In [ ]:
import sys
sys.path.append("../src")

import warnings
warnings.filterwarnings("ignore")

import json
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

In [2]:
from Classes.cognitive_model_agents import MODELS
from Classes.cognitive_model_agents import *
from Classes.parameter_recovery import ParameterFit

In [3]:
# Create paths for data and results
data_folder = Path('..', 'data', 'human')
folder_2P = Path('..', 'reports', 'MLE')
folder_2P.mkdir(parents=True, exist_ok=True)
best_fit_file_2P = Path(folder_2P, f'best_fit_2P.json')

folder_all = Path('..', 'reports', 'MLE')
folder_all.mkdir(parents=True, exist_ok=True)
best_fit_file_all = Path(folder_all, f'best_fit_all.json')

## Fit to 2-player data

In [4]:
#Load 2P data

file_name = '2-player-UR.csv'
file = data_folder / Path(file_name)
print(f'Loading data from {file}...')
data = pd.read_csv(file)
data.head()

Loading data from ..\data\human\2-player-UR.csv...


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,source,treatment,group,round,player,choice,score,attendance,ac_score,threshold,num_players,room,decision
0,0,0,0,0,0,human,score + grid,Grupo-0001,1,405186428721292,0,0,1,25,0.5,2,Grupo-0001,0
1,1,1,1,1,1,human,score + grid,Grupo-0001,1,728608683559397,1,1,1,25,0.5,2,Grupo-0001,1
2,2,2,2,2,2,human,score + grid,Grupo-0001,2,405186428721292,1,1,1,25,0.5,2,Grupo-0001,1
3,3,3,3,3,3,human,score + grid,Grupo-0001,2,728608683559397,0,0,1,25,0.5,2,Grupo-0001,0
4,4,4,4,4,4,human,score + grid,Grupo-0001,3,405186428721292,0,0,1,25,0.5,2,Grupo-0001,0


In [21]:
# Create optimization hyperparameters
hyperparameters = {
    'init_points':4,
    'n_iter':8
}

#Initialize records
best_fit = {
    'model_name': list(),
    'deviance': list(),
    'AIC': list(),
    'free_parameters': list()
}

In [22]:
# MODELS is a dictionary with the free parameters for 
# each model and comes from Classes.cognitive_model_agents
for model_name, model in tqdm(MODELS.items(), desc='Models '):
    # if model_name not in ['QAttendance', 'QFairness']:
    # 	continue
    print(f'Fitting data to model {model_name}...')
    best_fit['model_name'].append(model_name)
    print('Creating parameter recovery class...')
    pf = ParameterFit(
        agent_class=model["class"],
        model_name=model_name,
        free_parameters=model["free_parameters"],
        data=data,
        optimizer_name='bayesian'
    )
    print('Running bayesian optimizer...')
    res = pf.get_optimal_parameters(hyperparameters)
    best_fit['deviance'].append(-res["target"])
    best_fit['AIC'].append(2*len(model["free_parameters"]) - 2*res["target"])
    best_fit['free_parameters'].append(res["params"])
print(best_fit)

Models :   0%|          | 0/17 [00:00<?, ?it/s]

Fitting data to model Random...
Creating parameter recovery class...
pbounds: {'go_prob': (0, 1)}
Running bayesian optimizer...
|   iter    |  target   |  go_prob  |
-------------------------------------
| 1         | -3.361e+0 | 0.417     |
| 2         | -3.38e+03 | 0.7203    |
| 3         | -1.811e+0 | 0.0001144 |
| 4         | -3.849e+0 | 0.3023    |
| 5         | -1.367e+0 | 1.0       |
| 6         | -3.318e+0 | 0.4325    |
| 7         | -3.153e+0 | 0.6016    |
| 8         | -3.152e+0 | 0.6001    |
| 9         | -3.167e+0 | 0.5197    |
| 10        | -3.241e+0 | 0.6689    |
| 11        | -3.144e+0 | 0.5596    |
| 12        | -3.588e+0 | 0.3538    |
Fitting data to model WSLS...
Creating parameter recovery class...
pbounds: {'inverse_temperature': (4, 32), 'go_drive': (0, 1), 'wsls_strength': (0, 2)}
Running bayesian optimizer...
|   iter    |  target   | go_drive  | invers... | wsls_s... |
-------------------------------------------------------------
| 1         | -1.045e+0 | 0.417 

In [23]:
list_lens = [len(val) for key, val in best_fit.items()]
max_len = max(list_lens)
assert(np.all(x == max_len for x in list_lens))
with open(best_fit_file_2P, 'w') as f:
    json.dump(best_fit, f)
print(f'Model recovery data writen to file {best_fit_file_2P}')

Model recovery data writen to file ..\reports\MLE\best_fit_2P.json


## Fit to all player data

In [24]:
# Load data into a dataframe
file_names = [
	'2-player-UR.csv',
	'3-player-IU.csv',
	'4-player-IU.csv',
	'5-player-IU.csv',
	'6-player-IU.csv',
	'7-player-IU.csv',
	'8-player-IU.csv',
	'9-player-IU.csv',
	'11-player-IU.csv',
	'12-player-IU.csv',
]
df_list = list()
for file_name in file_names:
	file = data_folder / Path(file_name)
	print(f'Loading data from {file}...')
	df = pd.read_csv(file)
	df_list.append(df)
data = pd.concat(df_list, ignore_index=True)
data.head()

Loading data from ..\data\human\2-player-UR.csv...
Loading data from ..\data\human\3-player-IU.csv...
Loading data from ..\data\human\4-player-IU.csv...
Loading data from ..\data\human\5-player-IU.csv...
Loading data from ..\data\human\6-player-IU.csv...
Loading data from ..\data\human\7-player-IU.csv...
Loading data from ..\data\human\8-player-IU.csv...
Loading data from ..\data\human\9-player-IU.csv...
Loading data from ..\data\human\11-player-IU.csv...
Loading data from ..\data\human\12-player-IU.csv...


,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,source,treatment,group,round,player,...,score,attendance,ac_score,threshold,num_players,room,decision,timeup,time,date
0,0.0,0.0,0.0,0.0,0.0,human,score + grid,Grupo-0001,1.0,405186428721292,...,0.0,1.0,25.0,0.5,2,Grupo-0001,0.0,NaN,NaN,NaN
1,1.0,1.0,1.0,1.0,1.0,human,score + grid,Grupo-0001,1.0,728608683559397,...,1.0,1.0,25.0,0.5,2,Grupo-0001,1.0,NaN,NaN,NaN
2,2.0,2.0,2.0,2.0,2.0,human,score + grid,Grupo-0001,2.0,405186428721292,...,1.0,1.0,25.0,0.5,2,Grupo-0001,1.0,NaN,NaN,NaN
3,3.0,3.0,3.0,3.0,3.0,human,score + grid,Grupo-0001,2.0,728608683559397,...,0.0,1.0,25.0,0.5,2,Grupo-0001,0.0,NaN,NaN,NaN
4,4.0,4.0,4.0,4.0,4.0,human,score + grid,Grupo-0001,3.0,405186428721292,...,0.0,1.0,25.0,0.5,2,Grupo-0001,0.0,NaN,NaN,NaN


In [ ]:
#Initialize records
best_fit = {
    'model_name': list(),
    'deviance': list(),
    'AIC': list(),
    'free_parameters': list()
}

# MODELS is a dictionary with the free parameters for 
# each model and comes from Classes.cognitive_model_agents
for model_name, model in MODELS.items():
    # if model_name not in ['QAttendance', 'QFairness']:
    # 	continue
    print(f'Fitting data to model {model_name}...')
    best_fit['model_name'].append(model_name)
    print('Creating parameter recovery class...')
    pf = ParameterFit(
        agent_class=model["class"],
        model_name=model_name,
        free_parameters=model["free_parameters"],
        data=data,
        optimizer_name='bayesian'
    )
    print('Running bayesian optimizer...')
    res = pf.get_optimal_parameters(hyperparameters)
    best_fit['deviance'].append(-res["target"])
    best_fit['AIC'].append(2*len(model["free_parameters"]) - 2*res["target"])
    best_fit['free_parameters'].append(res["params"])
print(best_fit)
with open(best_fit_file_all, 'w') as f:
    json.dump(best_fit, f)
print(f'Model recovery data writen to file {best_fit_file_all}')

Fitting data to model Random...
Creating parameter recovery class...
pbounds: {'go_prob': (0, 1)}
Running bayesian optimizer...
|   iter    |  target   |  go_prob  |
-------------------------------------
| 1         | -2.345e+0 | 0.417     |
| 2         | -2.32e+04 | 0.7203    |
| 3         | -1.279e+0 | 0.0001144 |


In [ ]:
list_lens = [len(val) for key, val in best_fit.items()]
max_len = max(list_lens)
assert(np.all(x == max_len for x in list_lens))
with open(best_fit_file_all, 'w') as f:
    json.dump(best_fit, f)
print(f'Model recovery data writen to file {best_fit_file_all}')